In [3]:
import os
import sys
sys.path.insert(0, os.path.abspath('./dtl'))
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import math
from node import Node
from myC45 import Tree as tree_myC45
from myID3 import Tree as tree_myID3

In [4]:
# read data tennis
print('Data Tennis')
data_tennis = pd.read_csv("./data/play_tennis.csv")
print(data_tennis.head())

#make tennis tree
data_tennis = data_tennis.drop('day', axis=1)
tree_tennis = tree_myID3(data_tennis, 'play', use_info_gain=True)
root_tennis = tree_tennis.make_tree()
tree_tennis.print_tree(root_tennis, 0, 2)

Data Tennis
  day   outlook  temp humidity    wind play
0  D1     Sunny   Hot     High    Weak   No
1  D2     Sunny   Hot     High  Strong   No
2  D3  Overcast   Hot     High    Weak  Yes
3  D4      Rain  Mild     High    Weak  Yes
4  D5      Rain  Cool   Normal    Weak  Yes
-------tree-------
outlook
|--(Sunny)-->humidity
|            |--(High)-->{class : No}
|            |--(Normal)-->{class : Yes}
|--(Overcast)-->{class : Yes}
|--(Rain)-->wind
|            |--(Weak)-->{class : Yes}
|            |--(Strong)-->{class : No}


In [5]:
#read data iris
load, target = load_iris(return_X_y=True)
iris_data = pd.DataFrame(load, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
iris_data['label'] = pd.Series(target)
print(iris_data.head())

#make iris tree
tree_iris = tree_myC45(iris_data, 'label')
root_iris = tree_iris.make_tree()
tree_iris.print_tree(root_iris, 0, 2)

   sepal_length  sepal_width  petal_length  petal_width  label
0           5.1          3.5           1.4          0.2      0
1           4.9          3.0           1.4          0.2      0
2           4.7          3.2           1.3          0.2      0
3           4.6          3.1           1.5          0.2      0
4           5.0          3.6           1.4          0.2      0
-------tree-------
petal_length
|--(<=2.45)-->{class : 0}
|--(>2.45)-->petal_width
|            |--(<=1.7)-->sepal_length
|            |            |--(<=7.1)-->sepal_width
|            |            |            |--(<=2.8)-->{class : 1}
|            |            |            |--(>2.8)-->{class : 1}
|            |            |--(>7.1)-->{class : 2}
|            |--(>1.7)-->sepal_length
|            |            |--(<=5.9)-->sepal_width
|            |            |            |--(<=3.1)-->{class : 2}
|            |            |            |--(>3.1)-->{class : 1}
|            |            |--(>5.9)-->{class : 2}


In [6]:
def export_tree_myID3(node, depth, space, array):
    if (depth == 0):
        dash = ''
    else:
        dash = ('--'*space)*depth + '>' + node.parent_value + ' '
        
    if(node.is_leaf):
        output = dash + '{' + str(node.leaf_value) + '}'
    else:
        output = dash + '[' + node.split_attr + ']' 
    
    # print(output)
    array.append(output)
    
    depth += 1
    for child in node.childs:
        export_tree_myID3(child, depth, space, array)

In [7]:
def export_tree_C45(node, depth, space, array):
    if (depth == 0):
        dash = ''
    else:
        dash = ('--'*space)*depth + node.parent_value + ' '
        
    if(node.is_leaf):
        output = dash + '{' + str(node.leaf_value) + '}'
    else:
        output = dash + '[' + node.split_attr + ']' 
    
    # print(output)
    array.append(output)
    
    depth += 1
    for child in node.childs:
        export_tree_C45(child, depth, space, array)

In [8]:
# export file for ID3 tree
model_ID3 = []
export_tree_myID3(root_tennis, 0, 2, model_ID3)
text_file = open("tree_ID3.txt", "w")
for L in model_ID3:
    text_file.write(L+"\n")
text_file.close()

In [9]:
# export file for C45 tree
model_C45 = []
export_tree_C45(root_iris, 0, 2, model_C45)
text_file2 = open("tree_C45.txt", "w")
for L in model_C45:
    text_file2.write(L+"\n")
text_file2.close()

In [16]:
def importID3Model(path,data,target_attr):
    f = open(path,"r")
    lines = f.readlines()

    root_split_attr = ''.join((filter(lambda x: x not in ['[', ']','\n'], lines.pop(0))))
    root = Node(data,root_split_attr,target_attr)
    tree = tree_myID3(data,target_attr,root_value=root)
    print(root_split_attr)

    i_last=0
    spaces = []
    spaces.append(lines[0].count('-'))
    for line in lines:
        if (line.count('-')==spaces[i_last]):
            pass #do nothing  
        elif (line.count('-')>spaces[i_last]):
            spaces.append(line.count('-'))
            i_last += 1
        else: #kecil
            break
    print(spaces)

    i = 0
    curSpace = lines[0].count('-')
    curRoot = root
    prevRoot = root
    for line in lines:
        pre_str = ''.join((filter(lambda x: x not in ['-', '>','\n'], line)))
        pre_str = pre_str.split(' ')
        parent_value = pre_str[0]
        print(parent_value)
        split_or_leaf = pre_str[1]
        print(split_or_leaf)
        #split
        if(split_or_leaf[0]=='['):
            split = ''.join((filter(lambda x: x not in ['[', ']'], split_or_leaf)))
            child = Node(data,split,target_attr,parent_value=parent_value)
            print("split : "+split)
        #leaf
        elif(split_or_leaf[0]=='{'):
            leaf = ''.join((filter(lambda x: x not in ['{', '}'], split_or_leaf)))
            child = Node(data,'',target_attr,parent_value=parent_value,is_leaf=True,leaf_value=leaf);
            print("leaf : "+ leaf)
        #case add child
        if (line.count('-') == spaces[0]):
            root.add_child(child)
            prevRoot = child
            curRoot = child
            i = 0
        elif (line.count('-') > spaces[i]):
            curRoot.add_child(child)
            curRoot = child
        elif (line.count('-') == spaces[i]):
            prevRoot.add_child(child)
    return tree;

In [17]:
ID3_tree = importID3Model("tree_ID3.txt", data_tennis, "play")
ID3_tree.print_tree

outlook
[4, 8]
Sunny
[humidity]


NameError: name 'node' is not defined